In [5]:
!pip uninstall -y theano
!pip uninstall -y theano-pymc

Found existing installation: Theano-PyMC 1.1.2
Uninstalling Theano-PyMC-1.1.2:
  Successfully uninstalled Theano-PyMC-1.1.2


In [6]:
!pip install theano-pymc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 22.3 MB/s eta 0:00:0000:0100:01
  Created wheel for theano-pymc: filename=Theano_PyMC-1.1.2-py3-none-any.whl size=1529963 sha256=3e61e505f1f3759121e71064005fa4bb9627e361ff367d3d36b2d319d037e8f1
  Stored in directory: /Users/emilyziyixiao/Library/Caches/pip/wheels/c2/da/87/4e3e2d14772741721d4ebe739c16bcf10ca3c6348f740aa852
Successfully built theano-pymc
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.2
    Uninstalling numpy-1.24.2:
      Successfully uninstalled numpy-1.24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.10.0 requires libclang>=13.0.0, which is not installed.
tensorflow 2.10.0 requires tensorflow-io-gcs-filesystem>=0.23

In [8]:
!pip install arviz

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.10.0 requires libclang>=13.0.0, which is not installed.
tensorflow 2.10.0 requires tensorflow-io-gcs-filesystem>=0.23.1, which is not installed.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


In [11]:
!pip install theano-pymc==1.1.2 pymc3==3.11.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.4/869.4 kB 12.5 MB/s eta 0:00:0000:01
  Attempting uninstall: pymc3
    Found existing installation: pymc3 3.11.4
    Uninstalling pymc3-3.11.4:
      Successfully uninstalled pymc3-3.11.4


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import arviz as az
import pymc as pm
from patsy import dmatrix

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import BSpline
import arviz as az

# Read the data
data = pd.read_csv('ESM2023_20230428_5m.csv')
data['DateTime'] = pd.to_datetime(data['DateTime'])
data['Close'] = np.log(data['Close'])

# Generate the knot locations
num_knots = 9
knots = np.linspace(data['DateTime'].min(), data['DateTime'].max(), num_knots)

# Compute the basis functions
def compute_basis_functions(x, knots, degree=3):
    num_basis = len(knots) + degree - 1
    basis = np.zeros((x.shape[0], num_basis))
    for i in range(num_basis):
        coeffs = np.zeros(num_basis)
        coeffs[i] = 1
        basis[:, i] = BSpline(knots, coeffs, degree)(x)
    return basis

x = (data['DateTime'] - data['DateTime'].min()) / (data['DateTime'].max() - data['DateTime'].min())
x = x.values
y = data['Close'].values
basis_functions = compute_basis_functions(x, knots)

# Fit the Bayesian cubic spline model
with pm.Model() as model:
    w = pm.Normal("w", mu=0, sd=10, shape=basis_functions.shape[1])
    a = pm.Normal("a", mu=0, sd=10)
    sigma = pm.HalfNormal("sigma", sd=10)

    mu = a + tt.dot(basis_functions, w)
    y_obs = pm.Lognormal("y_obs", mu=mu, sd=sigma, observed=y)

    trace = pm.sample(2000, tune=2000, chains=2, target_accept=0.9)

# Plot the fitted curve and 90% credible interval
plt.figure(figsize=(12, 6))
x_pred = np.linspace(0, 1, 500)
basis_pred = compute_basis_functions(x_pred, knots)
y_pred = np.exp(np.dot(basis_pred, np.mean(trace['w'], axis=0)) + np.mean(trace['a'], axis=0))
y_pred_low = np.exp(np.dot(basis_pred, np.percentile(trace['w'], 5, axis=0)) + np.percentile(trace['a'], 5, axis=0))
y_pred_high = np.exp(np.dot(basis_pred, np.percentile(trace['w'], 95, axis=0)) + np.percentile(trace['a'], 95, axis=0))

plt.plot(data['Timestamp'], np.exp(y), 'o', markersize=4, label='Data', alpha=0.5)
plt.plot(data['Timestamp'].min() + (data['Timestamp'].max() - data['Timestamp'].min()) * x_pred, y_pred, label='Fitted curve')
plt.fill_between(data['Timestamp'].min() + (data['Timestamp'].max() - data['Timestamp'].min()) * x_pred, y_pred_low, y_pred_high, alpha=0.3, label='90% credible interval')

for knot in knots:
    plt.axvline(knot, color='gray', linestyle='--', label='Knots' if knot == knots[0] else None)

plt.xlabel('Timestamp')
plt.ylabel('Close')
plt.legend()
plt.title('Bayesian Cubic Spline Fit')
plt.show()

# Plot trace diagnostics using ArviZ
az.plot_trace(trace)
plt.show()

TypeError: unsupported operand type(s) for *: 'Timestamp' and 'float'

In [15]:
data

,Unnamed: 0,DateTime,Open,High,Low,Close,Volume,Count
0,2023-04-27 18:00:00,2023-04-27 18:00:00,4151.00,4153.00,4148.25,4151.00,3035,921
1,2023-04-27 18:05:00,2023-04-27 18:05:00,4151.00,4152.25,4150.25,4151.75,1872,507
2,2023-04-27 18:10:00,2023-04-27 18:10:00,4151.75,4152.25,4149.50,4149.75,1301,350
3,2023-04-27 18:15:00,2023-04-27 18:15:00,4149.75,4151.00,4149.25,4150.25,744,230
4,2023-04-27 18:20:00,2023-04-27 18:20:00,4150.25,4151.00,4149.00,4149.25,728,243
...,...,...,...,...,...,...,...,...
271,2023-04-28 16:35:00,2023-04-28 16:35:00,4190.25,4191.00,4190.25,4190.50,1435,239
272,2023-04-28 16:40:00,2023-04-28 16:40:00,4190.50,4191.00,4189.00,4189.50,3001,338
273,2023-04-28 16:45:00,2023-04-28 16:45:00,4189.75,4191.00,4189.25,4190.00,1168,301
274,2023-04-28 16:50:00,2023-04-28 16:50:00,4190.00,4191.00,4189.75,4191.00,4745,303


In [17]:
!pip install cmdstanpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 1.8 MB/s eta 0:00:0000:01


In [18]:
import cmdstanpy
cmdstanpy.install_cmdstan()

Installing CmdStan version: 2.32.0
Install directory: /Users/emilyziyixiao/.cmdstan
Download successful, file: /var/folders/zr/nth0p58d0lncg_nv89hbpzsm0000gn/T/tmplzk8s5wd
Extracting distribution
Unpacked download as cmdstan-2.32.0
Building version cmdstan-2.32.0, may take several minutes, depending on your system.


19:29:38 - cmdstanpy - WARNING - CmdStan installation failed.
Command "make build" failed
Command ['make', 'build', '-j1']
	error during processing No such file or directory


False

In [21]:
!sudo apt-get update
!sudo apt-get install build-essential -asdfghjkl;'


Password:
sudo: a password is required
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import BSpline
from cmdstanpy import CmdStanModel, cmdstan_path
import os

# Read the data
data = pd.read_csv('ESM2023_20230428_5m.csv')
data['DateTime'] = pd.to_datetime(data['DateTime'])
data['Close'] = np.log(data['Close'])

# Generate the knot locations
num_knots = 9
timestamps_numeric = (data['DateTime'] - data['DateTime'].min()) / np.timedelta64(1, 's')
knots_numeric = np.linspace(timestamps_numeric.min(), timestamps_numeric.max(), num_knots)
knots = data['DateTime'].min() + pd.to_timedelta(knots_numeric, unit='s')

# Compute the basis functions
def compute_basis_functions(x, knots, degree=3):
    num_basis = len(knots) + degree - 1
    basis = np.zeros((x.shape[0], num_basis))
    for i in range(num_basis):
        coeffs = np.zeros(num_basis)
        coeffs[i] = 1
        basis[:, i] = BSpline(knots, coeffs, degree)(x)
    return basis

x = (data['DateTime'] - data['DateTime'].min()) / (data['DateTime'].max() - data['DateTime'].min())
x = x.values
y = data['Close'].values
basis_functions = compute_basis_functions(x, knots)

# Save the data in a dictionary
data_dict = {
    "N": len(y),
    "K": basis_functions.shape[1],
    "y": y,
    "X": basis_functions
}

# Write the Stan model
stan_model_code = """
functions {
  matrix cubic_spline_basis(vector x, vector knots) {
    int N = rows(x);
    int K = rows(knots);
    matrix[N, K + 3] B;
    vector[N] x_shifted = x - knots[1];
    for (i in 1:N) {
      for (j in 1:K) {
        if (x[i] >= knots[j] && x[i] < knots[j + 1]) {
          B[i, j] = pow(x_shifted[i], 3) - 4 * pow(x_shifted[i] - knots[j], 3) * (x[i] > knots[j]);
        } else if (x[i] >= knots[j + 1] && x[i] < knots[j + 2]) {
          B[i, j] = pow(knots[j + 2] - knots[1], 3) - 4 * pow(x_shifted[i] - knots[j + 1], 3);
        } else {
          B[i, j] = 0;
        }
      }
      B[i, K + 1] = x_shifted[i] * x_shifted[i];
      B[i, K + 2] = x_shifted[i];
      B[i, K + 3] = 1;
    }
    return B;
  }
}
data {
  int<lower=0> N;
  vector[N] x;
  vector[N] y;
  int<lower=0> K;
  vector[K] knots;
}
transformed data {
  matrix[N, K + 3] B = cubic_spline_basis(x, knots);
}
parameters {
  vector[K + 3] w;
  real a;
  real<lower=0> sigma;
}
model {
  y ~ lognormal(a + B * w, sigma);
}
"""

with open("stan_model.stan", "w") as f:
    f.write(stan_model_code)

stan_model = CmdStanModel(stan_file="stan_model.stan")

# Fit the model
fit = stan_model.sample(data=data_dict, iter_sampling=2000, iter_warmup=2000, chains=2)

# Get the posterior samples
w_samples = fit.stan_variable("w")
a_samples = fit.stan_variable("a")
sigma_samples = fit.stan_variable("sigma")

# Compute the fitted curve and 90% credible interval
x_pred = np.linspace(0, 1, 500)
basis_pred = compute_basis_functions(x_pred, knots)
y_pred = np.exp(np.dot(basis_pred, w_samples.mean(axis=0)) + a_samples.mean())
y_pred_low = np.exp(np.dot(basis_pred, np.percentile(w_samples, 5, axis=0)) + np.percentile(a_samples, 5))
y_pred_high = np.exp(np.dot(basis_pred, np.percentile(w_samples, 95, axis=0)) + np.percentile(a_samples,
                                                                                              95))

# Plot the fitted curve and 90% credible interval
plt.figure(figsize=(12, 6))
plt.plot(data['DateTime'], np.exp(y), 'o', markersize=4, label='Data', alpha=0.5)
time_range = data['DateTime'].max() - data['DateTime'].min()
plt.plot(data['DateTime'].min() + pd.to_timedelta(time_range.total_seconds() * x_pred, unit='s'), y_pred, label='Fitted curve')
plt.fill_between(data['DateTime'].min() + pd.to_timedelta(time_range.total_seconds() * x_pred, unit='s'), y_pred_low, y_pred_high, alpha=0.3, label='90% credible interval')


for knot in knots:
    plt.axvline(knot, color='gray', linestyle='--', label='Knots' if knot == knots[0] else None)

plt.xlabel('Timestamp')
plt.ylabel('Close')
plt.legend()
plt.title('Bayesian Cubic Spline Fit')
plt.show()

# Print summary statistics of the fit
print(fit.summary())

20:19:57 - cmdstanpy - INFO - compiling stan file /Users/emilyziyixiao/DataspellProjects/AdvDsFinance/Final/stan_model.stan to exe file /Users/emilyziyixiao/DataspellProjects/AdvDsFinance/Final/stan_model


ValueError: Failed to compile Stan model '/Users/emilyziyixiao/DataspellProjects/AdvDsFinance/Final/stan_model.stan'. Console:
x86_64-apple-darwin13.4.0-clang++ -march=core2 -mtune=haswell -mssse3 -ftree-vectorize -fPIC -fPIE -fstack-protector-strong -O2 -pipe -stdlib=libc++ -fvisibility-inlines-hidden -std=c++14 -fmessage-length=0 -isystem /opt/anaconda3/include -std=c++1y -Wno-unknown-warning-option -Wno-tautological-compare -Wno-sign-compare -D_REENTRANT -Wno-ignored-attributes      -I stan/lib/stan_math/lib/tbb_2020.3/include    -O3 -I src -I stan/src -I stan/lib/rapidjson_1.1.0/ -I lib/CLI11-1.9.1/ -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.4.0 -I stan/lib/stan_math/lib/boost_1.78.0 -I stan/lib/stan_math/lib/sundials_6.1.1/include -I stan/lib/stan_math/lib/sundials_6.1.1/src/sundials -D_FORTIFY_SOURCE=2 -mmacosx-version-min=10.9 -isystem /opt/anaconda3/include    -DBOOST_DISABLE_ASSERTS          -c -MT stan/src/stan/model/model_header.hpp.gch -MT stan/src/stan/model/

In [29]:
!pip install tensorflow_probability

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 24.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.4/115.4 kB 3.5 MB/s eta 0:00:00


In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
from scipy.interpolate import BSpline

tfd = tfp.distributions
tfb = tfp.bijectors
tfpl = tfp.layers

# Read the data
data = pd.read_csv('ESM2023_20230428_5m.csv')
data['DateTime'] = pd.to_datetime(data['DateTime'])
data['Close'] = np.log(data['Close'])

# Generate the knot locations
num_knots = 9
knots = np.linspace(data['DateTime'].min(), data['DateTime'].max(), num_knots)

# Compute the basis functions
def compute_basis_functions(x, knots, degree=3):
    num_basis = len(knots) + degree - 1
    basis = np.zeros((x.shape[0], num_basis))
    for i in range(num_basis):
        coeffs = np.zeros(num_basis)
        coeffs[i] = 1
        basis[:, i] = BSpline(knots, coeffs, degree)(x)
    return basis

x = (data['DateTime'] - data['DateTime'].min()) / (data['DateTime'].max() - data['DateTime'].min())
x = x.values
y = data['Close'].values
basis_functions = compute_basis_functions(x, knots)

# Build and fit the Bayesian cubic spline model
def build_model(input_shape):
    model = tf.keras.Sequential([
        tfpl.DenseVariational(
            input_shape=input_shape,
            units=1,
            make_prior_fn=lambda t: tfd.MultivariateNormalDiag(loc=tf.zeros(13), scale_diag=tf.ones(13)),
            make_posterior_fn=lambda t: tfd.MultivariateNormalDiag(
                loc=tf.Variable(tf.zeros(13)),
                scale_diag=tfp.util.TransformedVariable(tf.ones(13), bijector=tfb.Softplus())
            ),
            kl_weight=1 / x.shape[0]
        )
    ])
    return model

model = build_model(input_shape=(13,))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse')
model.fit(basis_functions, y, epochs=1000, verbose=0)

# Plot the fitted curve and 90% credible interval
x_pred = np.linspace(0, 1, 500)
basis_pred = compute_basis_functions(x_pred, knots)

samples = model(basis_pred).sample(500).numpy()
y_pred = np.exp(samples.mean(axis=0))
y_pred_low = np.exp(np.percentile(samples, 5, axis=0))
y_pred_high = np.exp(np.percentile(samples, 95, axis=0))

plt.figure(figsize=(12, 6))
plt.plot(data['DateTime'], np.exp(y), 'o', markersize=4, label='Data', alpha=0.5)
plt.plot(data['DateTime'].min() + (data['DateTime'].max() - data['DateTime'].min()) * x_pred, y_pred, label='Fitted curve')
plt.fill_between(data['DateTime'].min() + (data['DateTime'].max() - data['DateTime'].min()) * x_pred, y_pred_low, y_pred_high, alpha=0.3, label='90% credible interval')

for knot in knots:
    plt.axvline(knot, color='gray', linestyle='--', label='Knots


SyntaxError: unterminated string literal (detected at line 71) (2101376661.py, line 71)